## Scrapy 사용

**저는 colab에서 작업했습니다.**

pwd, cd명령어를 활용해서 시작할 위치를 잡아주시면 됩니다.

- pwd : 현재 위치
- cd [이동할 폴더명] : 위치 이동

In [ ]:
# !pip install scrapy

In [15]:
# cd 명령어를 사용해서 scrapy를 만들 장소(폴더)로 이동합니다.
!pwd

/content


In [11]:
# scrapy 초기화(생성) startproject [이름 입력]
# scrapy 프로젝트 만드는 과정입니다. 이름은 자유!
!scrapy startproject navernews

New Scrapy project 'navernews', using template directory '/usr/local/lib/python3.9/dist-packages/scrapy/templates/project', created in:
    /content/navernews

You can start your first spider with:
    cd navernews
    scrapy genspider example example.com


In [33]:
# spider폴더로 이동해야 하므로 cd사용해서 이동해줍니다. cd ./폴더명/폴더명/spiders
%cd ./navernews/navernews/spiders

/content/navernews/navernews/spiders
/content/navernews/navernews/spiders


In [34]:
# 크롤링을 진행할 메인 py 파일을 초기화(생성)
#genspider [이름] 도메인 < 이름은 원하시는 이름으로 대신 위에서 생성한 프로젝트 이름과 겹치면 안됩니다. 도메인 초기화는 저도 아직 잘 모르겠네요.. 이걸 해줘야한다고 합니다.
!scrapy genspider navernewsspider news.naver.com

Created spider 'navernewsspider' using template 'basic' in module:
  navernews.spiders.navernewsspider


**여기까지 하시면 크롤링을 위한 밑작업은 끝났습니다. 밑에는 자동으로 생성된 파일들을 수정해서 크롤링하는 과정입니다.**

In [ ]:
# 폴더이름/폴더이름/items.py

"""
items.py는 scrapy 프로젝트를 만들면 기본으로 만들어지는 곳입니다.
크롤링 한 정보를 담아둘 수납공간이라고 생각하시면 됩니다. 
기존에 크롤링했을때 사용한 변수라고 생각하시면 될 것 같아요.
"""

import scrapy


class NavernewsItem(scrapy.Item):
    source = scrapy.Field() # 신문사
    category_1 = scrapy.Field() # 카테고리1
    category_2 = scrapy.Field() # 카테고리2
    title = scrapy.Field() # 제목
    article = scrapy.Field() # 기사
    reg_date = scrapy.Field() #등록 날짜
    writer = scrapy.Field() # 기자


In [ ]:
# 폴더이름/폴더이름/settings.py

"""
저는 이부분이 처음에 True로 설정되어 있어서 계속 파싱을 못했습니다. 
ROBOTSTEXT_OBEY 값이 True일 경우 사이트에서 설정한 값을 확인하고 로봇이 접근을 못하게 했다면 접근을 안한다고 하네요.
실제로 위 링크 들어가보면 네이버 뉴스는 로봇 접근을 막아놨습니다. (Disallow : \ <이게 전부 막는다는 뜻이라고 합니다.)
그런데 이걸 False로 바꿔주면 로봇이여도 접근이 가능하다고 합니다. 근데 가급적이면 막아놓은 곳은 사용 안하는게 좋다고 하네요. 자세한건 저도 좀 더 찾아봐야할 것 같아요.
(사이트다마다 url끝에 /robots.txt를 붙여서 들어가서 확인하실 수 있습니다. ex. https://news.naver.com/robots.txt)
"""
ROBOTSTXT_OBEY = False


"""
가져와야할 기사가 많아서 딜레이를  계속 줄여봤습니다. 저는 0.001로 했습니다.

저는 USER-agent부분을 fake로 사용했습니다.
!pip install scrapy-fake-useragent<< 실행하시고 밑에 DOWNLOADER_MIDDLEWARES 추가해주시면 정상 적용됩니다.
"""

DOWNLOAD_DELAY = 0.001

DOWNLOADER_MIDDLEWARES = {
    'scrapy.contrib.downloadermiddleware.useragent.UserAgentMiddleware': None,
    'scrapy.contrib.downloadermiddleware.retry.RetryMiddleware': None,
    'scrapy_fake_useragent.middleware.RandomUserAgentMiddleware': 400,
    'scrapy_fake_useragent.middleware.RetryUserAgentMiddleware': 401,
}

In [ ]:
# 폴더이름/폴더이름/spiders/크롤링 진행할 메인 파일.py

import scrapy
from navernews.items import NavernewsItem #위에서 사용할 Item의 공간을 미리 만들어놨으니 사용하기 위해서 불러오는 과정
from datetime import datetime, timedelta

import time
import re

# 각자 진행하는 카테고리에 맞게 수정하시면 됩니다. 코드는 직접 찾으셔야해요
category = {
  #경제
  '101' : ['258','259','260','261','253','310','771'],
  #IT/과학
  '105' : ['226','227','228','229','230','283','731','732']
}

# 코드 매칭
category_dict = {
    '101' : '경제',
    '105' : 'IT/과학',

    #경제
    '259' : '금융',
    '258' : '증권',
    '260' : '부동산',
    '261' : '산업/제계',
    '262' : '글로벌 경제',
    '263' : '경제일반',
    '310' : '생활경제',
    '771' : '중기/벤처',

    #IT/과학
    '226' : '인터넷/SNS',
    '227' : '통신/뉴미디어',
    '228' : '과학/일반',
    '229' : '게임/리뷰',
    '230' : 'IT일반',
    '283' : '컴퓨터',
    '731' : '모바일',
    '732' : '보안/해킹'
    
}


# 크롤링 하는 클래스
class naverNewsUrlSpider(scrapy.Spider):
    name = "naverNewsUrlSpider"
    allowed_domains = ["news.naver.com"]
    start_urls = ["http://news.naver.com/"]
        
    # Scrapy로 크롤링을 실행시키면 이 함수가 실행됩니다. 
    def start_requests(self):

      # 탐색할 날짜 리스트를 먼저 생성합니다.
      start_date = datetime(2023, 2, 1)
      end_date = datetime(2023, 3, 16)
      dates = [start_date]
      while True:
          start_date = start_date + timedelta(days=1)
          dates.append(start_date)
          if start_date == end_date:
              break
      
      # 날짜 -> 카테고리1 -> 카테고리2
      for date in dates:
        detail_date = date.strftime('%Y%m%d') #  20230201 형태
        for main, sub_lst in category.items(): # 카테고리1 : 경제, IT/과학
          for sub in sub_lst: # 카테고리2
            url = f'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2={sub}&sid1={main}&date={detail_date}&page=1'
            yield scrapy.Request(url, self.url_parse, meta = {'page' : 1, 'urls' : [],  'main' : main, 'sub' : sub})
            
    # 보이지 않지만 현재 네이버 뉴스 -> 카테고리1 선택 화면입니다. 넘어온 상황
    def url_parse(self, response):
      # 10개의 기사가 출력되어있는 상황이니 url들을 가져옵니다.
      urls = response.xpath('//*[@id="main_content"]/div[2]/ul[1]/li/dl/dt[2]/a/@href').extract()

      # url 목록이 이전 페이지와 같다면 종료.(재귀)
      if response.meta.pop('urls') == urls:
          return
      
      # url 목록에 있는 기사들을 하나씩 가져오도록 하는 부분입니다.
      for url in urls:
        yield scrapy.Request(url, self.parse, meta={**response.meta})

      # 다음페이지(재귀)
      page = response.meta.pop('page')
      target_url = re.sub('page\=\d+', f'page={page+1}', response.url)
      yield scrapy.Request(url=target_url, callback=self.parse_url, meta={**response.meta, 'page': page+1, 'urls':urls})


    # 기사 본문 페이지를 읽는 부분. 원하시는 형태로 저장하시면 됩니다.
    def parse(self, response):

      item = NavernewsItem()
      item['title'] = response.xpath('//*[@id="title_area"]/span/text()').extract()
      item['source'] = response.xpath('//*[@id="ct"]/div[1]/div[1]/a/img[1]/@title').extract()
      item['category_1'] = category_dict[response.meta.pop('main')]
      item['category_2'] = category_dict[response.meta.pop('sub')]
      item['article'] = response.xpath('//*[@id="dic_area"]/text()').extract()
      item['reg_date'] = response.xpath('//*[@id="ct"]/div[1]/div[3]/div[1]/div/span/text()').extract()[0].split()[0]

      yield item


In [3]:
%cd ./navernews/navernews/spiders

/content/navernews/navernews/spiders


In [ ]:
!pip install scrapy-fake-useragent

In [ ]:
# URL정보를 가져오는 클래스를 호출합니다.
# scrapy crawl [클래스 이름] < 이 부분은 spiders폴더에서 진행했던 py폴더에서 class [이름] 이렇게 작성했던 이름을 써주시면 됩니다.
# [-o '저장할 이름.csv -t csv] 이부분은 csv파일로 저장하기 위해서 추가했습니다.
!scrapy crawl naverNewsUrlSpider -o test.csv -t csv

In [10]:
import pandas as pd

len(pd.read_csv('test.csv'))

114645

### 결과물

경제와 IT의 모든 기사 크롤링하는데 25분 걸렸습니다.

![img](https://user-images.githubusercontent.com/119479455/226164572-cc7820a0-cb2a-4214-8b6d-233d83dc9422.png)